<a href="https://colab.research.google.com/github/Trickshotblaster/nn-practices/blob/main/TrigramLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math

In [2]:
import requests

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'

response = requests.get(url)

with open('file.txt', 'wb') as f:
  f.write(response.content)

In [3]:
with open('file.txt', 'r') as data:
  dataset = data.readlines()

with open('file.txt', 'r') as data:
  raw_data = data.read()


In [4]:
raw_data[:10]

'First Citi'

In [5]:
dataset[:10]

['First Citizen:\n',
 'Before we proceed any further, hear me speak.\n',
 '\n',
 'All:\n',
 'Speak, speak.\n',
 '\n',
 'First Citizen:\n',
 'You are all resolved rather to die than to famish?\n',
 '\n',
 'All:\n']

In [6]:
vocab = set(raw_data)

In [7]:
vocab_len = len(vocab)

In [8]:
vocab_len

65

In [9]:
stoi = {char:i for i, char in enumerate(vocab)}

In [10]:
itos = {i:char for i, char in enumerate(vocab)}

In [11]:
import torch

In [12]:
import torch.nn.functional as functional

In [13]:
# make dataset
xs, ys = [], []
for data in dataset:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    xs.append([stoi[ch1], stoi[ch2]])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

2085234

In [14]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([130, 65])

In [15]:
xenc = functional.one_hot(xs[0], num_classes=vocab_len).float().flatten()
xenc @ ws

tensor([ 0.7750, -0.0269,  0.0971, -0.5464,  1.4381,  0.3443, -0.3498,  0.8244,
        -0.9096,  0.2255, -0.5896,  1.3723, -2.8381, -1.1601, -1.9528, -1.3536,
         1.4057, -0.8329,  0.1905, -1.7405, -0.5451,  0.1654,  2.0011, -0.4239,
        -2.4662,  1.7987, -0.1563, -2.1269,  0.8009,  1.2958,  0.6566, -1.0248,
        -1.0976, -0.5375,  3.1270,  1.4490,  0.2425,  0.3338,  0.9359, -1.6811,
         0.7757,  0.3484,  0.5065, -0.3559, -1.1660,  1.5092,  0.8692, -3.5372,
        -0.2465, -1.3641, -0.8557,  1.0563,  0.6269,  0.8126, -1.1735, -0.1024,
        -2.0307, -0.7345,  0.1607, -0.1908,  0.3549,  0.7368,  2.6795, -0.7835,
        -0.5290], grad_fn=<SqueezeBackward4>)

In [16]:
(xenc @ ws).shape

torch.Size([65])

In [28]:
num_epochs = 20
lr = 50

In [18]:
torch.randn((3, 1))

tensor([[-2.5794],
        [-1.1740],
        [ 0.1214]])

In [29]:
for epoch in range(num_epochs):
    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean()
    print(loss.item())

    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr

2.381587505340576
2.3765952587127686
2.372049331665039
2.367884635925293
2.364048719406128
2.3604984283447266
2.357198476791382
2.354119300842285
2.351235866546631
2.348527193069458
2.345975875854492
2.3435652256011963
2.3412821292877197
2.3391153812408447
2.33705472946167
2.3350911140441895
2.333216428756714
2.331423759460449
2.329706907272339
2.3280603885650635


In [32]:
xs[:10]

tensor([[35,  1],
        [ 1, 33],
        [33, 20],
        [20, 39],
        [39, 63],
        [63, 40],
        [40,  1],
        [ 1, 39],
        [39,  1],
        [ 1, 11]])

In [33]:
def forward(xs):
  xenc = functional.one_hot(xs, num_classes=65).float().flatten(start_dim=1)
  logits = xenc @ ws
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  outchar = itos[torch.multinomial(probs, num_samples=1).item()]
  return outchar

def prompt(ins):
  out = ins
  for char in range(500):
    last_two = ins[-2:]
    xs = torch.tensor([[stoi[last_two[0]], stoi[last_two[1]]]])
    outs = forward(xs)
    out += outs
  return out

prompt("Rom")

"Rom eb   eeeeeiKX hSy.ee pbeye ; eevsb, e ,y\nqe b e? y.,e:   eeeeS s me eaeeLeSeeyxpbe eeebei eeey    y eyeea yy   ,;  e T y eX  y.,saeee'y' qq beaye pee eseee  \nge y,ebyes eyS a    em o pe,ek  3  se.. , ee eepweX.a,e A;beaDleKebf eaeey\nbeee Faek e  ob. ,  expl edeeyey,yeey d,mRe  ? ;eeb e ye,oyaee yebaQye ee. eeeei ek eD XueeLe  , ee eee  y e'ebXyi p y  ,y R eaeee.  ees ,b y.yey a .ee;peee XYy Qe  e      empT, eeepe e ee&\n ,eme   aee.e.Jeee 'e,bXye,eb Kp f.ycX e  e eee nee e:;ye ye eeu:  eeT yeee "